In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 10 epochs
# learning rate = 0.0001
# dimension reduction algo

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Display
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import numpy as np
import os
import time
import pickle
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras import backend as K
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model, load_model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Input
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.optimizers import Adam, RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from skimage.transform import resize
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import *
from keras.models import Model
from keras.applications.vgg19 import VGG19
import os
from xgboost import XGBClassifier
#os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3'
import pickle
from datetime import datetime
import cv2
import math
import numpy as np
import os
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns
import PIL
import torch
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader, Subset
from torch.utils.tensorboard import SummaryWriter
from torch.optim import lr_scheduler
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from torch.nn import init
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torch import nn
import time, copy, argparse
import multiprocessing
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score, accuracy_score
from torch import FloatTensor
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score,recall_score,precision_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import svm
import random
import functools
import joblib
import pickle
import os
import cv2
from glob import glob
from random import shuffle


### X_RAY

In [ ]:
def data_prep_x_ray():
  root_folder = '/kaggle/input/covid-xray-dataset'
  diction = {}

  

  dir_images = []
  dir_label = []
  
  count_covid = 0
  count_non_covid = 0
    
  for subdir in os.listdir(root_folder):
          print('subdir',os.path.join(root_folder, subdir))
          for file_count, file_name in enumerate( sorted(glob(os.path.join(root_folder, subdir)+ '/*'),key=len) ):
              
              img = cv2.imread(file_name)
              img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
              dir_images.append(img)
              dir_label.append(subdir)
              if subdir =='NonCOVID':
                  count_non_covid +=1
                  if count_non_covid ==1500:
                    break
              elif subdir == 'COVID':
                  count_covid +=1
                  if count_covid == 1500:
                    break
              

  diction_train = {}
  diction_test = {}
  dir_label_shuf_train = []
  dir_images_shuf_train = []
  index_shuf = list(range(len(dir_label)))
  shuffle(index_shuf)

  train_shuf = index_shuf[0:int(len(index_shuf))]


  for i in train_shuf:
      dir_label_shuf_train.append(dir_label[i])
      dir_images_shuf_train.append(dir_images[i])    
          
  diction_train['X_tr'] = dir_images_shuf_train
  diction_train['y_tr'] = dir_label_shuf_train


        
        
  print(len(diction_train['X_tr']))
  ##print(diction_train['y_tr'])
  print(diction_train['X_tr'][0].shape)


 


  with open('training.pickle', 'wb') as handle:
      pickle.dump(diction_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

  

  nrows=224 #nrows = 299 
  ncolumns=224 #ncolumns = 299 #Inception
  channels=1

  batch_size=16
  epochs=1
  # Number of classes
  num_cpu = multiprocessing.cpu_count()
  num_classes = 2
  torch.manual_seed(8)
  torch.cuda.manual_seed(8)
  np.random.seed(8)
  random.seed(8)




  device = torch.device("cpu")

  dbfile = open('training.pickle', 'rb')      
  db = pickle.load(dbfile) 
  X_train,y_train = db['X_tr'],db['y_tr']

  X = []
  #X_train=np.reshape(np.array(X_train),[len(X_train),])
  for img in list(range(0,len(X_train))):
    if X_train[img].ndim>=3:
        X.append(np.moveaxis(cv2.resize(X_train[img][:,:,:3], (nrows,ncolumns),interpolation=cv2.INTER_CUBIC), -1, 0))
    else:
        smimg= cv2.cvtColor(X_train[img],cv2.COLOR_GRAY2RGB)
        X.append(np.moveaxis(cv2.resize(smimg, (nrows,ncolumns),interpolation=cv2.INTER_CUBIC), -1, 0))
    
    if y_train[img]=='COVID':
        y_train[img]=1
    elif y_train[img]=='NonCOVID' :
        y_train[img]=0
    else:
        continue

  x = np.array(X)
  y_train = np.array(y_train)


  outputs_all = []
  labels_all = []

  X_train, X_val, y_train, y_val = train_test_split(x, y_train, test_size=0.2, random_state=2)

  #base_model = tf.keras.applications.VGG19(weights = 'imagenet')
  #model = Model(inputs=base_model.input,outputs = base_model.get_layer('block5_pool').output)
  base_model = tf.keras.applications.ResNet50(weights = 'imagenet')
  model = Model(inputs=base_model.input,outputs = base_model.get_layer('avg_pool').output)
  #base_model = tf.keras.applications.DenseNet121(weights='imagenet')
  #model = Model(inputs=base_model.input,outputs = base_model.get_layer('avg_pool').output)
  #base_model = tf.keras.applications.InceptionResNetV2(weights='imagenet')
  #model = Model(inputs=base_model.input,outputs = base_model.get_layer('avg_pool').output)

  feature_dict={0:[],1:[]}
  feature_dict_val={0:[],1:[]}
  classes=['0','1']
  for j in range(len(X_train)):
    xx = X_train[j].transpose(2,1,0)
    xx = np.expand_dims(xx,axis=0)
  
    feature_dict[y_train[j]].append(model.predict(xx))


  for j in range(len(X_val)):
    xx = X_val[j].transpose(2,1,0)
    xx = np.expand_dims(xx,axis=0)
  
    feature_dict_val[y_val[j]].append(model.predict(xx))


  pickle.dump(feature_dict, open('features_till_flatten.pkl', "wb" ))
  pickle.dump(feature_dict_val, open('features_till_flatten_val.pkl', "wb" ))



  labels = []
  feature_list = []
  feature_dict=pickle.load(open('features_till_flatten.pkl','rb'))
  names = [0,1]
  for cat,features  in feature_dict.items():
      labels.extend([int(cat)] * len(features))
      feature_list.append(features)
  feature_list = np.concatenate(feature_list)
  feature_list1=[]
  for i in range(0,len(feature_list)):
      feature_list1.append(feature_list[i].ravel())


  labels2 = []
  feature_list_val = []
  feature_dict_val=pickle.load(open('features_till_flatten_val.pkl','rb'))
  names = [0,1]
  for cat,features  in feature_dict_val.items():
      labels2.extend([int(cat)] * len(features))
      feature_list_val.append(features)
  feature_list_val = np.concatenate(feature_list_val)
  feature_list2=[]
  for i in range(0,len(feature_list_val)):
      feature_list2.append(feature_list_val[i].ravel())

  scaler = StandardScaler().fit(feature_list1)
  rescaledX = scaler.transform(feature_list1)
  rescaledX_validation = scaler.transform(feature_list2)
  
  return rescaledX, rescaledX_validation, labels, labels2


In [ ]:
rescaledX_xray, rescaledX_validation_xray, labels_xray,labels2_xray = data_prep_x_ray()

# FSVM

In [ ]:
!pip install FuzzyClassificator

In [ ]:
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers
import pandas as pd

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
from sklearn import svm
import math

In [ ]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

def gaussian_kernel(x, y, sigma=10.0):
   # print(-linalg.norm(x-y)**2)
    x=np.asarray(x)
    y=np.asarray(y)
    return np.exp((-linalg.norm(x-y)**2) / (2 * (sigma ** 2)))

def gm(y_predict,y_test):
    test_min=0
    test_max=0
    pred_min=0
    pred_max=0
    y_test=np.asarray(y_test)
    for i in range(0,836):
        if(y_test[i]==1):
             test_min=test_min+1
        else:
             test_max=test_max+1
    print("y_test min",test_min)       
    print("y_test max",test_max)
    for i in range(0,836):
        if(y_predict[i]==1 and y_predict[i]==y_test[i]):
             pred_min=pred_min+1
        elif(y_predict[i]==-1 and y_predict[i]==y_test[i]):
             pred_max=pred_max+1
    print("y_pred min",pred_min)       
    print("y_pred max",pred_max)
    se=pred_min/test_min
    sp=pred_max/test_max
    print(se,sp)
    gm=math.sqrt(se*sp)
    print("GM",gm)

In [ ]:
!pip install cvxopt

In [ ]:
from cvxopt import matrix
class HYP_SVM(object):

    def __init__(self, kernel=gaussian_kernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
    def m_func(self, X_train,X_test, y):
        n_samples, n_features = X_train.shape 
        nt_samples, nt_features= X_test.shape
        self.K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                self.K[i,j] = gaussian_kernel(X_train[i], X_train[j])
               # print(K[i,j])
        X_train=np.asarray(X_train)
        X_test=np.asarray(X_test)
        K1 = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K1[i,j] = gaussian_kernel(X_train[i], X_train[j])
               # print(K[i,j])
        print(K1.shape)
        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        sv = a > 1e-5
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a_org=a
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        self.sv_yorg=y
        self.kernel = gaussian_kernel
        X_train=np.asarray(X_train)
        b = 0
        for n in range(len(self.a)):
            b += self.sv_y[n]
            b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        b /= len(self.a)
       # print(self.a_org[1])
        #print(self.a_org.shape,self.sv_yorg.shape,K.shape)
        w_phi=0
        total=0
        for n in range(len(self.a_org)):
            w_phi = self.a_org[n] * self.sv_yorg[n] * K1[n] 
        self.d_hyp=np.zeros(n_samples)
        for n in range(len(self.a_org)):
            self.d_hyp += self.sv_yorg[n]*(w_phi+b)
        func=np.zeros((n_samples))
        func=np.asarray(func)
        typ=1
        if(typ==1):
            for i in range(n_samples):
                func[i]=1-(self.d_hyp[i]/(np.amax(self.d_hyp[i])+0.000001))
        beta=0.2
        if(typ==2):
            for i in range(n_samples):
                func[i]=2/(1+beta*self.d_hyp[i])
        r_max=103/4074
        r_min=1
        self.m=func[0:103]*r_min
        print(self.m.shape)
        self.m=np.append(self.m,func[103:4177]*r_max)
        print(self.m.shape)
        
 ##############################################################################


    def fit(self, X_train,X_test, y):
        self.kernel = gaussian_kernel
        n_samples, n_features = X_train.shape 
        nt_samples, nt_features = X_test.shape
        # Gram matrix

        print(self.K.shape)

        P = cvxopt.matrix(np.outer(y,y) * self.K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1,n_samples))
        A = matrix(A, (1,n_samples), 'd') #changes done
        b = cvxopt.matrix(0.0)
        #print(P,q,A,b)
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        print(solution['status'])
        # Lagrange multipliers
        a = np.ravel(solution['x'])
        a_org = np.ravel(solution['x'])
        # Support vectors have non zero lagrange multipliers
        for i in range(n_samples):
            sv=np.logical_or(self.a_org <self.m, self.a_org > 1e-5)
        #print(sv.shape)
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X_train[sv]
        self.sv_y = y[sv]
        #print("%d support vectors out of %d points" % (len(self.a), n_samples))

        # Intercept
        self.b = 0
        for n in range(len(self.a)):
            self.b += self.sv_y[n]
            self.b -= np.sum(self.a * self.sv_y * self.K[ind[n],sv])
        self.b /= len(self.a)
        print(self.b)

        # Weight vector
        if self.kernel == gaussian_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        else :
            self.w = None        
        
    def project(self, X):
        if self.w is None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np.zeros(len(X))
            X=np.asarray(X)
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * gaussian_kernel(X[i], sv)
                y_predict[i] = s
              #  print(y_predict[i])
            return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))

In [ ]:


if __name__ == "__main__":
    import pylab as pl           
    def hyp_svm():
        
        clf = HYP_SVM(C=100.0)
        typ=2
        clf.m_func(X_train,X_test,labels_xray)
        clf.fit(X_train,X_test, labels_xray)
        y_predict = clf.predict(X_test)
        gm(y_predict,labels2_xray)
        correct = np.sum(y_predict == y_test)
        print("%d out of %d predictions correct" % (correct, len(y_predict)))
        print("Accuracy",correct/len(y_predict))

    hyp_svm()



In [ ]:
# import convex optimization package
from cvxopt import matrix as c_matrix
from cvxopt import solvers as c_solvers

# define guassian kernel (depends on distance)
# K(X1,X2) = exponent(-sigma||X1-X1||^2)

def gaussian_kernel(xg, yg, sigma=100.0):
    x=np.asarray(xg)
    y=np.asarray(yg)
    return np.exp((-np.linalg.norm(x-y)**2) / (2 * (sigma ** 2)))

# define FSVM object

class FSVM(object):
    
    #object initialisation
    def __init__(self, kernel = gaussian_kernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: 
            self.C = float(self.C)
    
    #Fuzzy membership function
    
    def m_func(self, X, y, type = 1):
        n_samples, n_featuers = X.shape
        
        # calculate gaussian kernels
        
        self.K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                self.K[i,j] = self.kernel(X[i], X[j])
        
        # defining variables for Quadratic problem
        P = c_matrix(np.outer(y,y) *self.K)
        q = c_matrix(np.ones(n_samples)*-1)
        A = c_matrix(y, (1, n_samples))
        A = c_matrix(A, (1, n_samples), 'd')
        b = c_matrix(0.0)
        
        if self.C is None:
            G = c_matrix(np.diag(np.ones(n_samples) * -1))
            h = c_matrix(np.zeros(n_samples))
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = c_matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = c_matrix(np.hstack((tmp1, tmp2)))  
        
        # Solve the QP
        
        self.solution = c_solvers.qp(P,q,G,h,A,b)
        print(self.solution['status'])                # check solution has been dfound
        
        #find Lagrange multipliers
        lm = np.ravel(self.solution['x'])
        lm_org = np.ravel(self.solution['x'])
        # Since svm have non legrange multipliers, assign non zero values
        supvec = lm > 1e-5
        index = np.arange(len(lm))[supvec]
        self.lm_org = lm
        self.lm = lm[supvec]         # number of support vectors
        self.supvec = X[supvec]
        self.supvec_y = y[supvec]
        self.supvec_yorg = y
        print("%d support vectors out of %d points" % (len(self.lm), n_samples))
        
        X = np.asarray(X)
        
        #calculate intercept 
        
        self.ic = 0
        for i in range(len(self.lm)):
            self.ic += self.supvec_y[i]
            self.ic -= np.sum(self.lm * self.supvec_y * self.K[index[i],supvec])
        self.ic /= len(self.lm)
        
        # calculate weight vector
        w_phi = 0
        total = 0
        for i in range(len(self.lm_org)):
            w_phi = self.lm_org[i] * self.supvec_yorg[i] * self.K[i]
        
        #hyperplane (to calculate value)
        self.hyp = np.zeros(n_samples)
        for i in range (len(self.lm_org)):
            self.hyp += self.supvec_yorg[i] * (w_phi+self.ic)
        
        # fuzzy membership functions - linear and exponential
        f_func = np.zeros((n_samples))
        f_func = np.asarray(f_func)
        beta = 0.8
        if type ==1:
            for i in range(n_samples):
                f_func[i] = 1 - self.hyp[i]/(self.hyp.max() + 0.00001)
        else:
            for i in range(n_samples):
                f_func[i] = 2/(1+beta*self.hyp[i])
        
        min_count =0
        maj_count = 0
        for i in range (len(y)):
            if y[i] == 0:
                min_count += 1;
            else:
                maj_count+=1;
        r_pos = 1
        r_neg = min_count/maj_count #minority-to-majority class ratio
        self.m = f_func[0:maj_count]*r_pos
        self.m = np.append(self.m, f_func[maj_count:]*r_neg)
        
    # FIT FUNCTION
        
    def fit(self, X, y, kernel = gaussian_kernel):
        self.m_func(X, y)
        self.kernel = kernel
        n_samples, n_features = X.shape
        
        # Gram matrix
        Kfsvm = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                Kfsvm[i,j] = gaussian_kernel(self.m[i], self.m[j])
        
        P = c_matrix(np.outer(y,y) * Kfsvm)
        q = c_matrix(np.ones(n_samples) * -1)
        A = c_matrix(y, (1,n_samples))
        A = c_matrix(A, (1,n_samples), 'd') #changes done
        b = c_matrix(0.0)
        
        if self.C is None:
            G = c_matrix(np.diag(np.ones(n_samples) * -1))
            h = c_matrix(np.zeros(n_samples))
            
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = c_matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = c_matrix(np.hstack((tmp1, tmp2)))
        
        fsolution = c_solvers.qp(P, q, G, h, A, b)
        print(fsolution['status'])
        lm = np.ravel(fsolution['x'])
        lm_org = np.ravel(fsolution['x'])
        # Support vectors have non zero lagrange multipliers
        #  Use m matrix calculated in m_func
        for i in range(n_samples):
            supvec=np.logical_or(lm_org <self.m, lm_org > 1e-5)
        
        index = np.arange(len(lm))[supvec]
        self.lm = lm[supvec]
        self.supvec = self.m[supvec]
        self.supvec_y = y[supvec]

        # Intercept
        self.ic = 0
        for i in range(len(self.lm)):
            self.ic += self.supvec_y[i]
            self.ic -= np.sum(self.lm * self.supvec_y * Kfsvm[index[i],supvec])
        self.ic /= len(self.lm)

        # Weight vector
        self.w = None        
        
    def project(self, X):
        y_predict = np.zeros(len(X))
        X=np.asarray(X)
        for i in range(len(X)):
            s = 0
            for lm, supvec_y, supvec in zip(self.lm, self.supvec_y, self.supvec):
                s += lm * supvec_y * gaussian_kernel(X[i], supvec)
            y_predict[i] = s
        return y_predict + self.ic

    def predict(self, X):
        return np.sign(self.project(X))

In [ ]:
labels_xray = np.array(labels_xray)

In [ ]:
type(rescaledX_xray)

In [ ]:
clf = FSVM(C = 1)
clf.fit(rescaledX_xray, labels_xray)
y_predfsvm = clf.predict(rescaledX_validation_xray)

In [ ]:
labels2_xray = np.array(labels2_xray)

In [ ]:
tp = np.sum(y_predfsvm == labels2_xray)
fp = 0
for i in range(len(labels2_xray)):
    if y_predfsvm[i] == 1 and labels2_xray[i] != 1:
        fp += 1
print("%d out of %d predictions correct" % (tp, len(labels2_xray)))
print("Accuracy",tp/len(labels2_xray))
print("Precision", tp/(tp+fp))

In [ ]:
X_train.shape

In [ ]:
X_train = np.array(rescaledX_xray.astype(int))

In [ ]:
X_test = np.array(rescaledX_validation_xray.astype(int))

In [ ]:
import FuzzyClassificator

In [ ]:
mod = FuzzyClassificator()

# Various Classifiers

In [ ]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB

clf = [svm.SVC(probability=True),svm.SVC(kernel="rbf",probability=True),RandomForestClassifier(),KNeighborsClassifier(),AdaBoostClassifier(),GaussianNB(),XGBClassifier()]

dic_xray={}
for i in clf:
    dic_xray[i]={}
for i in clf:
    dic_xray[i]['Accuracy']=[]
    dic_xray[i]['f1_score']=[]
    dic_xray[i]['Recall']=[]
    
    dic_xray[i]['Precision']=[]
    dic_xray[i]['Sensitivity']=[]
    dic_xray[i]['Specificity']=[]

for i in clf:
  model_clf = i
  pred_prob_train = []
  pred_prob_test = []
  model_clf.fit(rescaledX_xray,labels_xray)

  pred = model_clf.predict(rescaledX_xray)
  pred_proba_train = model_clf.predict_proba(rescaledX_xray)
  pred_prob_train.append(pred_proba_train)
  pred_proba_test = model_clf.predict_proba(rescaledX_validation_xray)
  pred_prob_test.append(pred_proba_test)
  pred_val =  model_clf.predict(rescaledX_validation_xray)
  #print(classification_report(labels2,pred_val))
  cc = confusion_matrix(labels2_xray,pred_val)
  print(i)
  print('\n')
  print('Accuracy :  ',  accuracy_score(labels2_xray, pred_val)*100)
  print('f1_score :  ', f1_score(labels2_xray, pred_val)*100)
  print('Recall :   ', recall_score(labels2_xray, pred_val)*100)
  print('Precission :  ', precision_score(labels2_xray, pred_val)*100)
  print('Sensiticity : ',(cc[1][1]/(cc[1][1]+cc[1][0]))*100 )
  print('Specificity : ',(cc[0][0]/(cc[0][0]+cc[0][1]))*100 )
  dic_xray[i]['Accuracy'].append(accuracy_score(labels2_xray, pred_val)*100)
  dic_xray[i]['f1_score'].append(f1_score(labels2_xray, pred_val)*100)
  dic_xray[i]['Recall'].append(recall_score(labels2_xray,pred_val)*100)
  dic_xray[i]['Precision'].append(precision_score(labels2_xray,pred_val)*100)
  dic_xray[i]['Sensitivity'].append((cc[1][1]/(cc[1][1]+cc[1][0]))*100) 
  dic_xray[i]['Specificity'].append((cc[0][0]/(cc[0][0]+cc[0][1]))*100)
  print('*'*20)
  print('\n')

In [ ]:
dic_xray

# Nueral Network

In [ ]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
# load the dataset
rescaledX_xray = np.array(rescaledX_xray)
labels_xray = np.array(labels_xray)
# define the keras model
model = Sequential()
model.add(Dense(16, input_dim=(1536), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(rescaledX_xray, labels_xray, epochs=150, batch_size=10)
# evaluate the keras model
_, accuracy = model.evaluate(rescaledX_xray, labels_xray)
print('Accuracy: %.2f' % (accuracy*100))

pred_nerual = model.predict_classes(rescaledX_validation_xray)
pred_nerual = pred_nerual[:,0]
cc = confusion_matrix(labels2_xray,pred_nerual)
print('Accuracy :  ',  accuracy_score(labels2_xray, pred_nerual)*100)
print('f1_score :  ', f1_score(labels2_xray, pred_nerual)*100)
print('Recall :   ', recall_score(labels2_xray, pred_nerual)*100)
print('Precission :  ', precision_score(labels2_xray, pred_nerual)*100)
print('Sensiticity : ',(cc[1][1]/(cc[1][1]+cc[1][0]))*100 )
print('Specificity : ',(cc[0][0]/(cc[0][0]+cc[0][1]))*100 )
#dic['neural']={}
#dic['neural']['Accuracy']=accuracy_score(labels2, pred_val)*100
#dic['neural']['f1_score']=f1_score(labels2, pred_val)*100
#dic['neural']['Recall']=recall_score(labels2,pred_val)*100
#dic['neural']['Precision']=precision_score(labels2,pred_val)*100
#dic['neural']['Sensitivity']=(cc[1][1]/(cc[1][1]+cc[1][0]))*100 
#dic['neural']['Specificity']=(cc[0][0]/(cc[0][0]+cc[0][1]))*100 

In [ ]:
model_clf = clf[6]
pred_prob_train = []
pred_prob_test = []
model_clf.fit(rescaledX_xray,labels_xray)
for _ in range(10):
  pred = model_clf.predict(rescaledX_xray)
  pred_proba_train = model_clf.predict_proba(rescaledX_xray)
  pred_prob_train.append(pred_proba_train)
  pred_proba_test = model_clf.predict_proba(rescaledX_validation_xray)
  pred_prob_test.append(pred_proba_test)
  pred_val =  model_clf.predict(rescaledX_validation_xray)
  #print(classification_report(labels2,pred_val))
  #cc = confusion_matrix(labels2,pred_val)
pred_prob_zero= 0 
pred_prob_one= 0 
for i in range(10):
  pred_prob_zero += pred_prob_train[i][:,0]
  pred_prob_one += pred_prob_train[i][:,1]
pred_prob_zero /= 10
pred_prob_one /=10

log_value = []
for i,j in zip(pred_prob_zero, pred_prob_one):
  if i == 0 :
    log = -j*math.log(j,2)
  elif j==0:
    log = -i*math.log(i,2)
  elif i!=0 and j!=0:
    
      log = -(i * math.log(i,2) + j * math.log(j,2))
  log_value.append(log)

print(np.mean(log_value))

In [ ]:
pred_prob_zero

In [ ]:
base_model = tf.keras.applications.DenseNet121(weights = 'imagenet')

# GradCAM generation

In [ ]:
model_builder = tf.keras.applications.DenseNet121
img_size = (224, 224) #(299,299) Inception (224,224) ResNet50
preprocess_input = keras.applications.xception.preprocess_input
decode_predictions = keras.applications.xception.decode_predictions

last_conv_layer_name = 'relu'#'block5_conv4'#"conv_7b_ac"

# The local path to our target image
img_path = "/kaggle/input/covid-xray-dataset/COVID/COVID-1018.png"

display(Image(img_path))



In [ ]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()


In [ ]:
img_array = preprocess_input(get_img_array(img_path, size=img_size))

# Make model
model = model_builder(weights="imagenet")

# Remove last layer's softmax
#model.layers[-1].activation = None

# Print what the top predicted class is
preds = model.predict(img_array)
print("Predicted:", decode_predictions(preds, top=1)[0])

# Generate class activation heatmap


In [ ]:
def save_and_display_gradcam(img_path, heatmap, cam_path="imagenet_xray.jpg", alpha=0.4):
    # Load the original image
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    display(Image(cam_path))





In [ ]:
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

# Display heatmap
plt.matshow(heatmap)
plt.colorbar()
plt.savefig('heatmap_imagenet_xray.png')
plt.show()
save_and_display_gradcam(img_path, heatmap)

In [ ]:
alpha=0.4
cam_path="resnet_xray.jpg"
img = keras.preprocessing.image.load_img(img_path)
img = keras.preprocessing.image.img_to_array(img)

# Rescale heatmap to a range 0-255
heatmap = np.uint8(255 * heatmap)

# Use jet colormap to colorize heatmap
jet = cm.get_cmap("jet")

# Use RGB values of the colormap
jet_colors = jet(np.arange(256))[:, :3]
jet_heatmap = jet_colors[heatmap]

# Create an image with RGB colorized heatmap
jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

# Superimpose the heatmap on original image
superimposed_img = jet_heatmap * alpha + img
superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

# Save the superimposed image
superimposed_img.save(cam_path)

# Display Grad CAM
display(Image(cam_path))

In [ ]:
plt.imshow(jet_heatmap)

In [ ]:
plt.imshow(superimposed_img,cmap='RdBu')
plt.colorbar()

In [ ]:
model_builder = tf.keras.applications.ResNet50(weights='imagenet')
img_size = (224, 224) #(299,299) Inception
preprocess_input = keras.applications.xception.preprocess_input
decode_predictions = keras.applications.xception.decode_predictions

last_conv_layer_name = "conv5_block3_out"

# The local path to our target image
img_path = "/kaggle/input/covid-xray-dataset/COVID/COVID-1018.png"

In [ ]:
import csv
with open('Results_resnet_xray.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key, value in dic_xray.items():
       writer.writerow([key, value])

In [ ]:
probs=[]


In [ ]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense

# load the dataset
rescaledX = np.array(rescaledX_xray)
labels = np.array(labels_xray)
# define the keras model
model = Sequential()
model.add(Dense(16, input_dim=(2048), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='softmax'))
rescaledX_xray = np.array(rescaledX_xray)
labels = np.array(labels)
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(rescaledX, labels, epochs=10, batch_size=10)
# evaluate the keras model
probs, accuracy = model.evaluate(rescaledX_xray, labels_xray)
print('Accuracy: %.2f' % (accuracy*100))

pred_nerual = model.predict(rescaledX_validation_xray)
#pred_nerual = pred_nerual[:,0]
cc = confusion_matrix(labels2_xray,pred_nerual)
print('Accuracy :  ',  accuracy_score(labels2_xray, pred_nerual)*100)
print('f1_score :  ', f1_score(labels2_xray, pred_nerual)*100)
print('Recall :   ', recall_score(labels2_xray, pred_nerual)*100)
print('Precission :  ', precision_score(labels2_xray, pred_nerual)*100)
print('Sensiticity : ',(cc[1][1]/(cc[1][1]+cc[1][0]))*100 )
print('Specificity : ',(cc[0][0]/(cc[0][0]+cc[0][1]))*100 )
#dic['neural']={}
#dic['neural']['Accuracy']=accuracy_score(labels2, pred_val)*100
#dic['neural']['f1_score']=f1_score(labels2, pred_val)*100
#dic['neural']['Recall']=recall_score(labels2,pred_val)*100
#dic['neural']['Precision']=precision_score(labels2,pred_val)*100
#dic['neural']['Sensitivity']=(cc[1][1]/(cc[1][1]+cc[1][0]))*100 
#dic['neural']['Specificity']=(cc[0][0]/(cc[0][0]+cc[0][1]))*100 

In [ ]:
model.metrics_names

In [ ]:
for dire ,_,filename in os.walk('/kaggle/input/'):
    print(dire)

In [ ]:
root_folder='/kaggle/input/covid-xray-dataset'
for subdir in os.listdir(root_folder)[1:]:
    print('subdir',os.path.join(root_folder, subdir))
    

In [ ]:
count_covid = 0
count_non_covid = 0
for subdir in os.listdir(root_folder)[1:]:
          print(subdir)
          print('subdir',os.path.join(root_folder, subdir))
          for file_count, file_name in enumerate( sorted(glob(os.path.join(root_folder, subdir)+ '/*'),key=len) ):
              if subdir =='1NonCOVID':
                  count_non_covid +=1
                  if count_non_covid ==2500:
                        break
              elif subdir == '2COVID':
                  count_covid +=1
                  if count_covid == 2500:
                    break
              
          
print(count_non_covid)
print(count_covid)
print(count_non_covid+count_covid)

# CT Scan

In [ ]:
def data_prep():
  root_folder = '/kaggle/input/large-covid19-ct-slice-dataset/curated_data/curated_data'
  diction = {}

  

  dir_images = []
  dir_label = []
  
  count_covid = 0
  count_non_covid = 0
    
  for subdir in os.listdir(root_folder)[1:]:
          print('subdir',os.path.join(root_folder, subdir))
          for file_count, file_name in enumerate( sorted(glob(os.path.join(root_folder, subdir)+ '/*'),key=len) ):
              
              img = cv2.imread(file_name)
              img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
              dir_images.append(img)
              dir_label.append(subdir)
              if subdir =='1NonCOVID':
                  count_non_covid +=1
                  if count_non_covid ==1500:
                    break
              elif subdir == '2COVID':
                  count_covid +=1
                  if count_covid == 1500:
                    break
              

  diction_train = {}
  diction_test = {}
  dir_label_shuf_train = []
  dir_images_shuf_train = []
  index_shuf = list(range(len(dir_label)))
  shuffle(index_shuf)

  train_shuf = index_shuf[0:int(len(index_shuf))]
  

  for i in train_shuf:
      dir_label_shuf_train.append(dir_label[i])
      dir_images_shuf_train.append(dir_images[i])    
          
  diction_train['X_tr'] = dir_images_shuf_train
  diction_train['y_tr'] = dir_label_shuf_train



        
        
  print(len(diction_train['X_tr']))
  ##print(diction_train['y_tr'])
  print(diction_train['X_tr'][0].shape)


  

  with open('training.pickle', 'wb') as handle:
      pickle.dump(diction_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

  

  nrows=224
  ncolumns=224
  channels=1

  batch_size=16
  epochs=1
  # Number of classes
  num_cpu = multiprocessing.cpu_count()
  num_classes = 2
  torch.manual_seed(8)
  torch.cuda.manual_seed(8)
  np.random.seed(8)
  random.seed(8)




  device = torch.device("cpu")

  dbfile = open('training.pickle', 'rb')      
  db = pickle.load(dbfile) 
  X_train,y_train = db['X_tr'],db['y_tr']

  X = []
  #X_train=np.reshape(np.array(X_train),[len(X_train),])
  for img in list(range(0,len(X_train))):
    if X_train[img].ndim>=3:
        X.append(np.moveaxis(cv2.resize(X_train[img][:,:,:3], (nrows,ncolumns),interpolation=cv2.INTER_CUBIC), -1, 0))
    else:
        smimg= cv2.cvtColor(X_train[img],cv2.COLOR_GRAY2RGB)
        X.append(np.moveaxis(cv2.resize(smimg, (nrows,ncolumns),interpolation=cv2.INTER_CUBIC), -1, 0))
    
    if y_train[img]=='2COVID':
        y_train[img]=1
    elif y_train[img]=='1NonCOVID' :
        y_train[img]=0
    else:
        continue

  x = np.array(X)
  y_train = np.array(y_train)


  outputs_all = []
  labels_all = []

  X_train, X_val, y_train, y_val = train_test_split(x, y_train, test_size=0.2, random_state=2)

  #base_model = tf.keras.applications.VGG19(weights = 'imagenet')
  #model = Model(inputs=base_model.input,outputs = base_model.get_layer('block5_pool').output)
  base_model = tf.keras.applications.ResNet50(weights = 'imagenet')
  model = Model(inputs=base_model.input,outputs = base_model.get_layer('avg_pool').output)
  #base_model = tf.keras.applications.DenseNet121(weights='imagenet')
  #model = Model(inputs=base_model.input,outputs = base_model.get_layer('avg_pool').output)
  #base_model = tf.keras.applications.InceptionResNetV2(weights='imagenet')
  #model = Model(inputs=base_model.input,outputs = base_model.get_layer('avg_pool').output)

  feature_dict={0:[],1:[]}
  feature_dict_val={0:[],1:[]}
  classes=['0','1']
  for j in range(len(X_train)):
    xx = X_train[j].transpose(2,1,0)
    xx = np.expand_dims(xx,axis=0)
  
    feature_dict[y_train[j]].append(model.predict(xx))


  for j in range(len(X_val)):
    xx = X_val[j].transpose(2,1,0)
    xx = np.expand_dims(xx,axis=0)
  
    feature_dict_val[y_val[j]].append(model.predict(xx))


  pickle.dump(feature_dict, open('features_till_flatten.pkl', "wb" ))
  pickle.dump(feature_dict_val, open('features_till_flatten_val.pkl', "wb" ))



  labels = []
  feature_list = []
  feature_dict=pickle.load(open('features_till_flatten.pkl','rb'))
  names = [0,1]
  for cat,features  in feature_dict.items():
      labels.extend([int(cat)] * len(features))
      feature_list.append(features)
  feature_list = np.concatenate(feature_list)
  feature_list1=[]
  for i in range(0,len(feature_list)):
      feature_list1.append(feature_list[i].ravel())
  

  labels2 = []
  feature_list_val = []
  feature_dict_val=pickle.load(open('features_till_flatten_val.pkl','rb'))
  names = [0,1]
  for cat,features  in feature_dict_val.items():
      labels2.extend([int(cat)] * len(features))
      feature_list_val.append(features)
  feature_list_val = np.concatenate(feature_list_val)
  feature_list2=[]
  for i in range(0,len(feature_list_val)):
      feature_list2.append(feature_list_val[i].ravel())

  scaler = StandardScaler().fit(feature_list1)
  rescaledX = scaler.transform(feature_list1)
  rescaledX_validation = scaler.transform(feature_list2)
  
  return rescaledX, rescaledX_validation, labels, labels2 


In [ ]:
rescaledX, rescaledX_validation, labels,labels2 = data_prep()


In [ ]:
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

In [ ]:
clf = [svm.SVC(probability=True),svm.SVC(kernel="rbf",probability=True),RandomForestClassifier(),KNeighborsClassifier(),AdaBoostClassifier(),GaussianNB(),XGBClassifier()]

In [ ]:
dic={}

In [ ]:
for i in clf:
  model_clf = i
  dic[i]={}
  pred_prob_train = []
  pred_prob_test = []
  model_clf.fit(rescaledX,labels)

  pred = model_clf.predict(rescaledX)
  pred_proba_train = model_clf.predict_proba(rescaledX)
  pred_prob_train.append(pred_proba_train)
  pred_proba_test = model_clf.predict_proba(rescaledX_validation)
  pred_prob_test.append(pred_proba_test)
  pred_val =  model_clf.predict(rescaledX_validation)
  #print(classification_report(labels2,pred_val))
  cc = confusion_matrix(labels2,pred_val)
  print(i)
  print('\n')
  print('Accuracy :  ',  accuracy_score(labels2, pred_val)*100)
  print('f1_score :  ', f1_score(labels2, pred_val)*100)
  print('Recall :   ', recall_score(labels2, pred_val)*100)
  print('Precission :  ', precision_score(labels2, pred_val)*100)
  print('Sensiticity : ',(cc[1][1]/(cc[1][1]+cc[1][0]))*100 )
  print('Specificity : ',(cc[0][0]/(cc[0][0]+cc[0][1]))*100 )
  dic[i]['Accuracy']=accuracy_score(labels2, pred_val)*100
  dic[i]['f1_score']=f1_score(labels2, pred_val)*100
  dic[i]['Recall']=recall_score(labels2,pred_val)*100
  dic[i]['Precision']=precision_score(labels2,pred_val)*100
  dic[i]['Sensitivity']=(cc[1][1]/(cc[1][1]+cc[1][0]))*100 
  dic[i]['Specificity']=(cc[0][0]/(cc[0][0]+cc[0][1]))*100 
  print('*'*20)
  print('\n')

In [ ]:
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
# load the dataset
rescaledX = np.array(rescaledX)
labels = np.array(labels)
# define the keras model
model = Sequential()
model.add(Dense(16, input_dim=(2048), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit the keras model on the dataset
model.fit(rescaledX, labels, epochs=150, batch_size=10)
# evaluate the keras model
_, accuracy = model.evaluate(rescaledX, labels)
print('Accuracy: %.2f' % (accuracy*100))

pred_nerual = model.predict_classes(rescaledX_validation)
pred_nerual = pred_nerual[:,0]
cc = confusion_matrix(labels2,pred_nerual)
print('Accuracy :  ',  accuracy_score(labels2, pred_nerual)*100)
print('f1_score :  ', f1_score(labels2, pred_nerual)*100)
print('Recall :   ', recall_score(labels2, pred_nerual)*100)
print('Precission :  ', precision_score(labels2, pred_nerual)*100)
print('Sensiticity : ',(cc[1][1]/(cc[1][1]+cc[1][0]))*100 )
print('Specificity : ',(cc[0][0]/(cc[0][0]+cc[0][1]))*100 )
#dic['neural']={}
#dic['neural']['Accuracy']=accuracy_score(labels2, pred_val)*100
#dic['neural']['f1_score']=f1_score(labels2, pred_val)*100
#dic['neural']['Recall']=recall_score(labels2,pred_val)*100
#dic['neural']['Precision']=precision_score(labels2,pred_val)*100
#dic['neural']['Sensitivity']=(cc[1][1]/(cc[1][1]+cc[1][0]))*100 
#dic['neural']['Specificity']=(cc[0][0]/(cc[0][0]+cc[0][1]))*100 

In [ ]:
model_clf = clf[2]
pred_prob_train = []
pred_prob_test = []
model_clf.fit(rescaledX,labels)
for _ in range(10):
  pred = model_clf.predict(rescaledX)
  pred_proba_train = model_clf.predict_proba(rescaledX)
  pred_prob_train.append(pred_proba_train)
  pred_proba_test = model_clf.predict_proba(rescaledX_validation)
  pred_prob_test.append(pred_proba_test)
  pred_val =  model_clf.predict(rescaledX_validation)
  #print(classification_report(labels2,pred_val))
  #cc = confusion_matrix(labels2,pred_val)
pred_prob_zero= 0 
pred_prob_one= 0 
for i in range(10):
  pred_prob_zero += pred_prob_train[i][:,0]
  pred_prob_one += pred_prob_train[i][:,1]
pred_prob_zero /= 10
pred_prob_one /=10

log_value = []
for i,j in zip(pred_prob_zero, pred_prob_one):
  log = i * np.log(i) + j * np.log(j)
  log_value.append(log)
print(np.mean(log_value))

In [ ]:
#base_model = tf.keras.applications.VGG19(weights = 'imagenet')
#model = Model(inputs=base_model.input,outputs = base_model.get_layer('block5_pool').output)
#base_model = tf.keras.applications.ResNet50(weights = 'imagenet')
#model = Model(inputs=base_model.input,outputs = base_model.get_layer('avg_pool').output)
#base_model = tf.keras.applications.DenseNet121(weights='imagenet')
#model = Model(inputs=base_model.input,outputs = base_model.get_layer('avg_pool').output)
base_model = tf.keras.applications.InceptionResNetV2(weights='imagenet')
#model = Model(inputs=base_model.input,outputs = base_model.get_layer('avg_pool').output)
base_model.summary()

In [ ]:
model_builder = tf.keras.applications.InceptionResNetV2
img_size = (299, 299)
preprocess_input = keras.applications.xception.preprocess_input
decode_predictions = keras.applications.xception.decode_predictions

last_conv_layer_name = "conv_7b_ac"#'block5_conv4'#'relu'##"conv_7b_ac"#'conv5_block3_out'#"conv_7b_ac"

# The local path to our target image
img_path = "/kaggle/input/large-covid19-ct-slice-dataset/curated_data/curated_data/2COVID/13_Bjorke_100.png"


plt.savefig('normal.png')
display(Image(img_path))

In [ ]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()


In [ ]:
img_array = preprocess_input(get_img_array(img_path, size=img_size))

# Make model
model = model_builder(weights="imagenet")

# Remove last layer's softmax
#model.layers[-1].activation = None

# Print what the top predicted class is
preds = model.predict(img_array)
print("Predicted:", decode_predictions(preds, top=1)[0])

# Generate class activation heatmap


In [ ]:
def save_and_display_gradcam(img_path, heatmap, cam_path="imagenet.jpg", alpha=0.4):
    # Load the original image
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    display(Image(cam_path))





In [ ]:
heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)

# Display heatmap
plt.matshow(heatmap)
plt.colorbar()
plt.savefig('heatmap_imagenet.png')
plt.show()
save_and_display_gradcam(img_path, heatmap)